In [169]:
#!pip install simpledorff
import simpledorff as sf
import pandas as pd
import json
from glom import glom
pd.__version__

'1.3.5'

In [170]:

# load data using Python JSON module
with open('Jsonfiles/ch23_1.json','r') as f:
    data = json.loads(f.read())
# Flatten data, keep entry ID and person who completed the data
df_base_list = pd.json_normalize(data, record_path =['annotations', 'result'], meta = [
    'id',
    ['annotations', 'completed_by'],
    ['annotations', 'id'],
    ['data']
], record_prefix = '_',
    errors = 'ignore'
                                  )

# Anonymise annotator by giving them a number
df_base_list['annotations.completed_by'] = 1
df_base_list['_value.labels'] = df_base_list['_value.labels'].str[0]
df_base_list['_value.labels'] = df_base_list['_value.labels'].astype(str)

# Separate speakers and speeches
df_speakers = df_base_list[df_base_list['_value.labels'] == "Speaker"]
df_speeches = df_base_list[df_base_list['_value.labels'] != "Speaker"]

#Clean speakers dataframe by removing unused columns
series_speakers_name = df_speakers['_value.text']
series_speakers_id = df_speakers['_meta.text'].str[0]
df_speakers_final =  pd.concat([series_speakers_name, series_speakers_id], axis=1).reset_index()

# Drop all NaN values, or they'll cause issues
df_speeches = df_speeches.dropna(subset=['annotations.id'])
df_speakers_final = df_speakers_final.dropna(subset=['_meta.text'])

# Convert both columns to integer to merge
df_speeches['annotations.id'] = df_speeches['annotations.id'].astype(int)
df_speakers_final['_meta.text'] = df_speakers_final['_meta.text'].astype(int)

# Merge speakers and speeches dataframes based on matching IDs
df_final = pd.merge(df_speeches, df_speakers_final, how='inner', left_on=['annotations.id'], right_on=['_meta.text'])

# Remove speech lines and only keep those with emotions
df_final_emotions_1 = df_final[df_final['_from_name'] == 'emotion']

# TODO : Merge all 4 dataframes into one. Group_by?

# Turn emotions into numerical value
df_final_emotions_1 = pd.get_dummies(df_final_emotions_1, columns=['_value.labels'])

# Calculate Krippendorff' alpha
sf.calculate_krippendorffs_alpha_for_df(df_final_emotions_1)

TypeError: calculate_krippendorffs_alpha_for_df() missing 4 required positional arguments: 'df', 'experiment_col', 'annotator_col', and 'class_col'

In [121]:

# load data using Python JSON module
with open('Jsonfiles/ch23_2.json','r') as f:
    data = json.loads(f.read())
# Flatten data, keep entry ID and person who completed the data
df_base_list = pd.json_normalize(data, record_path =['annotations', 'result'], meta = [
    'id',
    ['annotations', 'completed_by'],
    ['annotations', 'id'],
    ['data']
], record_prefix = '_',
    errors = 'ignore'
                                  )
# Anonymise annotator by giving them a number
df_base_list['annotations.completed_by'] = 2

# Separate speakers and speeches
df_speakers = df_base_list[df_base_list['_value.labels'].map(lambda x: x[0]) == "Speaker"]
df_speeches = df_base_list[df_base_list['_value.labels'].map(lambda x: x[0]) != "Speaker"]

#Clean speakers dataframe by removing unused columns
series_speakers_name = df_speakers['_value.text']
series_speakers_id = df_speakers['_meta.text'].str[0]
df_speakers_final =  pd.concat([series_speakers_name, series_speakers_id], axis=1).reset_index()

# Drop all NaN values, or they'll cause issues
df_speeches = df_speeches.dropna(subset=['annotations.id'])
df_speakers_final = df_speakers_final.dropna(subset=['_meta.text'])

# Convert both columns to integer to merge
df_speeches['annotations.id'] = df_speeches['annotations.id'].astype(int)
df_speakers_final['_meta.text'] = df_speakers_final['_meta.text'].astype(int)

# Merge speakers and speeches dataframes based on matching IDs
df_final = pd.merge(df_speeches, df_speakers_final, how='inner', left_on=['annotations.id'], right_on=['_meta.text'])

# Remove speech lines and only keep those with emotions
df_final_emotions_2 = df_final[df_final['_from_name'] == 'emotion']

df_final_emotions_2


,_id,_from_name,_to_name,_type,_value.start,_value.end,_value.text_x,_value.labels,_meta.text_x,id,annotations.completed_by,annotations.id,data,index,_value.text_y,_meta.text_y
1,acYs295Xrp,emotion,text,labels,2,225,"AtrÃ©ide , nous savons combien tu l'emportes s...",[Calme],NaN,112,2,115,"{'text': 'â€” AtrÃ©ide , nous savons combien t...",2,Akhilleus,115
3,Et7lsZYNcJ,emotion,text,labels,2,295,"Qu'ils se lÃ¨vent , ceux qui veulent tenter ce...",[Ardeur],NaN,105,2,108,"{'text': 'â€” Qu'ils se lÃ¨vent , ceux qui veu...",7,Akhilleus,108
5,N_ILOjIBuH,emotion,text,labels,2,424,"Que deux guerriers , parmi les plus braves , e...",[Ardeur],NaN,102,2,105,"{'text': 'â€” Que deux guerriers , parmi les p...",10,le PÃ¨lÃ©ide,105
7,mCj9qRZJRE,emotion,text,labels,2,102,"Antilokhos , tu ne m'auras point louÃ© en vain...",[Ardeur],NaN,100,2,103,"{'text': 'â€” Antilokhos , tu ne m'auras point...",13,Akhilleus,103
9,GMGrUKWXUE,emotion,text,labels,2,348,"Je vous le dis Ã tous , et vous le voyez , am...",[Plaisir],NaN,98,2,101,"{'text': 'â€” Je vous le dis Ã tous , et vous...",17,Antilokhos,101
11,h4iJndMQ44,emotion,text,labels,2,136,"Malheur Ã moi ! certes , la dÃ©esse AthÃ¨nÃ¨ ...",[ColÃ¨re],NaN,96,2,99,"{'text': 'â€” Malheur Ã moi ! certes , la dÃ©...",21,Aias,99
13,QR_HvXVj-j,emotion,text,labels,2,66,"Exauce-moi , dÃ©esse ! soutiens-moi heureuseme...",[Ardeur],NaN,94,2,97,"{'text': 'â€” Exauce-moi , dÃ©esse ! soutiens-...",26,Odysseus,97
15,QudDEPz4q_,emotion,text,labels,2,62,"Qu'ils se lÃ¨vent , ceux qui veulent combattre...",[Ardeur],NaN,92,2,95,"{'text': 'â€” Qu'ils se lÃ¨vent , ceux qui veu...",29,Akhilleus,95
17,UL2rLKaLLM,emotion,text,labels,2,97,"Divin Laertiade , trÃ¨s sage Odysseus , enlÃ¨v...",[ColÃ¨re],NaN,87,2,90,"{'text': 'â€” Divin Laertiade , trÃ¨s sage Ody...",36,Aias,90
19,lDg7EJcze5,emotion,text,labels,2,535,"Qu'il vienne , celui qui veut emporter cette c...",[Ardeur],NaN,81,2,84,"{'text': 'â€” Qu'il vienne , celui qui veut em...",41,Ã‰pÃ©ios,84


In [122]:

# load data using Python JSON module
with open('Jsonfiles/ch23_3.json','r') as f:
    data = json.loads(f.read())
# Flatten data, keep entry ID and person who completed the data
df_base_list = pd.json_normalize(data, record_path =['annotations', 'result'], meta = [
    'id',
    ['annotations', 'completed_by'],
    ['annotations', 'id'],
    ['data']
], record_prefix = '_',
    errors = 'ignore'
                                  )
# Anonymise annotator by giving them a number
df_base_list['annotations.completed_by'] = 3

# Separate speakers and speeches
df_speakers = df_base_list[df_base_list['_value.labels'].map(lambda x: x[0]) == "Speaker"]
df_speeches = df_base_list[df_base_list['_value.labels'].map(lambda x: x[0]) != "Speaker"]

#Clean speakers dataframe by removing unused columns
series_speakers_name = df_speakers['_value.text']
series_speakers_id = df_speakers['_meta.text'].str[0]
df_speakers_final =  pd.concat([series_speakers_name, series_speakers_id], axis=1).reset_index()

# Drop all NaN values, or they'll cause issues
df_speeches = df_speeches.dropna(subset=['annotations.id'])
df_speakers_final = df_speakers_final.dropna(subset=['_meta.text'])

# Convert both columns to integer to merge
df_speeches['annotations.id'] = df_speeches['annotations.id'].astype(int)
df_speakers_final['_meta.text'] = df_speakers_final['_meta.text'].astype(int)

# Merge speakers and speeches dataframes based on matching IDs
df_final = pd.merge(df_speeches, df_speakers_final, how='inner', left_on=['annotations.id'], right_on=['_meta.text'])

# Remove speech lines and only keep those with emotions
df_final_emotions_3 = df_final[df_final['_from_name'] == 'emotion']

df_final_emotions_3


,_id,_from_name,_to_name,_type,_origin,_value.start,_value.end,_value.text_x,_value.labels,_meta.text_x,_parentID,id,annotations.completed_by,annotations.id,data,index,_value.text_y,_meta.text_y
1,J4U-29p-md,emotion,text,labels,manual,2,763,"Tu dors , et tu m'oublies , Akhilleus . Vivant...",[Tristesse],NaN,NaN,10,3,9,"{'text': 'â€” Tu dors , et tu m'oublies , Akhi...",2,l'Ã¢me du malheureux Patroklos,9
3,JD0gZue23X,emotion,text,labels,manual,2,192,Celui qui atteindra la colombe emportera les h...,[Ardeur],NaN,NaN,109,3,109,{'text': 'â€” Celui qui atteindra la colombe e...,7,le PÃ¨lÃ©ide,109
5,WPV8xt98c3,emotion,text,labels,manual,2,422,"Que deux guerriers , parmi les plus braves , e...",[Ardeur],NaN,NaN,102,3,102,"{'text': 'â€” Que deux guerriers , parmi les p...",11,Antilokhos,102
7,5zgFNQsoLj,emotion,text,labels,manual,2,100,"Antilokhos , tu ne m'auras point louÃ© en vain...",[Calme],NaN,NaN,100,3,100,"{'text': 'â€” Antilokhos , tu ne m'auras point...",14,Akhilleus,100
9,xCW45QCeo2,emotion,text,labels,manual,52,208,"maintenant et toujours , les immortels honoren...",[DÃ©goÃ»t],NaN,NaN,98,3,98,"{'text': 'â€” Je vous le dis Ã tous , et vous...",17,Antilokhos,98
11,59nW3Mgv4C,emotion,text,labels,manual,2,60,"Qu'ils se lÃ¨vent , ceux qui osent combattre p...",[Ardeur],NaN,NaN,85,3,85,"{'text': 'â€” Qu'ils se lÃ¨vent , ceux qui ose...",25,le PÃ¨lÃ©ide,85
13,MndBGZ1B3V,emotion,text,labels,manual,2,325,"Qu'il vienne , celui qui veut emporter cette c...",[Ardeur],NaN,NaN,81,3,81,"{'text': 'â€” Qu'il vienne , celui qui veut em...",29,"Ã‰pÃ©ios , fils de Panopeus",81
14,Ic5zDDba7t,emotion,text,labels,manual,326,533,"Mais , je le dis , et ma parole s'accomplira :...",[ColÃ¨re],NaN,NaN,81,3,81,"{'text': 'â€” Qu'il vienne , celui qui veut em...",29,"Ã‰pÃ©ios , fils de Panopeus",81
16,ij91mu5I7P,emotion,text,labels,manual,1,292,"Mon fils , certes , tu as bien parlÃ© . Ami ,...",[Tristesse],NaN,NaN,76,3,76,"{'text': 'â€” Mon fils , certes , tu as bien p...",30,NestÃ´r,76
17,PsQSdAHPrp,emotion,text,labels,manual,865,1035,"Tel j'Ã©tais autrefois , et maintenant de plus...",[Tristesse],NaN,NaN,76,3,76,"{'text': 'â€” Mon fils , certes , tu as bien p...",30,NestÃ´r,76


In [123]:

# load data using Python JSON module
with open('Jsonfiles/ch23_4.json','r') as f:
    data = json.loads(f.read())
# Flatten data, keep entry ID and person who completed the data
df_base_list = pd.json_normalize(data, record_path =['annotations', 'result'], meta = [
    'id',
    ['annotations', 'completed_by'],
    ['annotations', 'id'],
    ['data']
], record_prefix = '_',
    errors = 'ignore'
                                  )
# Anonymise annotator by giving them a number
df_base_list['annotations.completed_by'] = 4

# Separate speakers and speeches
df_speakers = df_base_list[df_base_list['_value.labels'].map(lambda x: x[0]) == "Speaker"]
df_speeches = df_base_list[df_base_list['_value.labels'].map(lambda x: x[0]) != "Speaker"]

#Clean speakers dataframe by removing unused columns
series_speakers_name = df_speakers['_value.text']
series_speakers_id = df_speakers['_meta.text'].str[0]
df_speakers_final =  pd.concat([series_speakers_name, series_speakers_id], axis=1).reset_index()

# Drop all NaN values, or they'll cause issues
df_speeches = df_speeches.dropna(subset=['annotations.id'])
df_speakers_final = df_speakers_final.dropna(subset=['_meta.text'])

# Convert both columns to integer to merge
df_speeches['annotations.id'] = df_speeches['annotations.id'].astype(int)
df_speakers_final['_meta.text'] = df_speakers_final['_meta.text'].astype(int)

# Merge speakers and speeches dataframes based on matching IDs
df_final = pd.merge(df_speeches, df_speakers_final, how='inner', left_on=['annotations.id'], right_on=['_meta.text'])

# Remove speech lines and only keep those with emotions
df_final_emotions_4 = df_final[df_final['_from_name'] == 'emotion']

df_final_emotions_4


,_id,_from_name,_to_name,_type,_origin,_value.start,_value.end,_value.text_x,_value.labels,_meta.text_x,id,annotations.completed_by,annotations.id,data,index,_value.text_y,_meta.text_y
1,72zUXkwxBO,emotion,text,labels,manual,2,225,"AtrÃ©ide , nous savons combien tu l'emportes s...",[Ardeur],NaN,112,4,83,"{'text': 'â€” AtrÃ©ide , nous savons combien t...",0,Akhilleus,83
3,2eG5aNICEQ,emotion,text,labels,manual,2,192,Celui qui atteindra la colombe emportera les h...,[Ardeur],NaN,109,4,81,{'text': 'â€” Celui qui atteindra la colombe e...,3,PÃ¨lÃ©ide,81
5,ddAu15XeSP,emotion,text,labels,manual,2,56,"Qu'ils se lÃ¨vent , ceux qui veulent tenter ce...",[Ardeur],NaN,105,4,79,"{'text': 'â€” Qu'ils se lÃ¨vent , ceux qui veu...",9,Akhilleus,79
6,I48U7dv9vQ,emotion,text,labels,manual,57,295,"Celui qui possÃ©dera ce disque , s'il a des ch...",[Ardeur],NaN,105,4,79,"{'text': 'â€” Qu'ils se lÃ¨vent , ceux qui veu...",9,Akhilleus,79
8,sVMXrI1DeF,emotion,text,labels,manual,2,424,"Que deux guerriers , parmi les plus braves , e...",[Calme],NaN,102,4,77,"{'text': 'â€” Que deux guerriers , parmi les p...",10,PÃ¨lÃ©ide,77
10,2h2Zas-uGr,emotion,text,labels,manual,2,102,"Antilokhos , tu ne m'auras point louÃ© en vain...",[Calme],NaN,100,4,75,"{'text': 'â€” Antilokhos , tu ne m'auras point...",13,Akhilleus,75
12,mhinFUfDao,emotion,text,labels,manual,2,348,"Je vous le dis Ã tous , et vous le voyez , am...",[Joie],NaN,98,4,73,"{'text': 'â€” Je vous le dis Ã tous , et vous...",16,Antilokhos,73
14,gamZNFLQnL,emotion,text,labels,manual,2,136,"Malheur Ã moi ! certes , la dÃ©esse AthÃ¨nÃ¨ ...",[ColÃ¨re],NaN,96,4,71,"{'text': 'â€” Malheur Ã moi ! certes , la dÃ©...",19,Aias,71
16,88UvpydG5G,emotion,text,labels,manual,2,66,"Exauce-moi , dÃ©esse ! soutiens-moi heureuseme...",[Ardeur],NaN,94,4,69,"{'text': 'â€” Exauce-moi , dÃ©esse ! soutiens-...",22,Odysseus,69
18,TI7yzlJZtQ,emotion,text,labels,manual,2,62,"Qu'ils se lÃ¨vent , ceux qui veulent combattre...",[Ardeur],NaN,92,4,67,"{'text': 'â€” Qu'ils se lÃ¨vent , ceux qui veu...",25,PÃ¨lÃ©ide,67


In [164]:
#df_total = pd.merge(df_final_emotions_1, df_final_emotions_2, on='data')



KeyError: "None of [Index(['_value.labels'], dtype='object')] are in the [columns]"